# isce2

## prepare data

In [ ]:
import os
os.chdir('/media/ly/文件/isce_cat')

In [ ]:
# os.chdir('/media/ly/文件/xsc/isce_stack/DEM/')
# !fixImageXml.py -i demLat_N30_N33_Lon_E101_E105.dem.wgs84 -f
# os.chdir('/media/ly/文件/xsc/isce_stack/')

In [ ]:
# !stackSentinel.py -h

In [ ]:
# !stackSentinel.py -s slc -d DEM/demLat_N30_N33_Lon_E101_E105.dem.wgs84 -o aux/aux_poeorb/ -a aux/aux_cal/ -b '31.569715 31.577582 103.412374 103.419046' -W interferogram -C geometry -r 15 -z 5 -f 0.5 -c 4
# !stackSentinel.py -s slc -d DEM/demLat_N30_N33_Lon_E101_E105.dem.wgs84 -o aux/aux_poeorb/ -a aux/aux_cal/ -b '31.569715 31.577582 103.412374 103.419046' -W interferogram -C geometry -r 7 -z 2 -f 0.5 -c 4
!stackSentinel.py -s zip -d dem/demLat_N27_N32_Lon_E100_E106.dem.wgs84 -o orbits/ -a aux_cal/ -b '30.780067 31.130474 102.589458 102.850073' -W interferogram -C geometry -r 20 -z 5 -f 0.5 -c 1

## run stack

In [ ]:
os.chdir('/media/ly/文件/isce_cat/run_files/')

In [ ]:
!./run_1_unpack_slc_topo_master

In [ ]:
!./run_2_average_baseline

In [ ]:
!./run_3_geo2rdr_resample

In [ ]:
!./run_4_extract_stack_valid_region

In [ ]:
!./run_5_merge_master_slave_slc

In [ ]:
!./run_6_merge_burst_igram

In [ ]:
!./run_7_filter_coherence

In [ ]:
!./run_8_unwrap

## prep files for MintPy

In [ ]:
# Copy necessary files processed by stackSentinel.py for MintPy
import os
import shutil
import sys


def copy_files(stack_dir, save_dir):
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    # copy baselines
    baselines_src = os.path.join(stack_dir, 'baselines')
    baselines_dst = os.path.join(save_dir, 'baselines')
    if os.path.isdir(baselines_src):
        print('copy baselines...')
        shutil.copytree(baselines_src, baselines_dst)
        print('done.')

    # copy reference
    master_src = os.path.join(stack_dir, 'reference')
    master_dst = os.path.join(save_dir, 'reference')
    if os.path.isdir(master_src):
        print('copy reference...')
        shutil.copytree(master_src, master_dst)
        print('done.')

    # copy merged/geom_reference
    merged_dst = os.path.join(save_dir, 'merged')
    os.mkdir(merged_dst)
    geom_master_src = os.path.join(stack_dir, 'merged', 'geom_reference')
    geom_master_dst = os.path.join(merged_dst, 'geom_reference')
    if os.path.isdir(geom_master_src):
        print('copy merged/geom_reference...')
        shutil.copytree(geom_master_src, geom_master_dst)
        print('done.')

    # copy merged/interferograms
    files = [
        'filt_fine.cor', 'filt_fine.cor.vrt', 'filt_fine.cor.xml',
        'filt_fine.unw', 'filt_fine.unw.vrt', 'filt_fine.unw.xml',
        'filt_fine.unw.conncomp', 'filt_fine.unw.conncomp.vrt',
        'filt_fine.unw.conncomp.xml'
    ]
    interferograms_dst = os.path.join(merged_dst, 'interferograms')
    os.mkdir(interferograms_dst)
    interferograms_src = os.path.join(stack_dir, 'merged', 'interferograms')
    ifgs = os.listdir(interferograms_src)
    if ifgs:
        for ifg in ifgs:
            sys.stdout.write(
                f"\rcopy merged/interferograms {ifgs.index(ifg) + 1}/{len(ifgs)}",
            )
            sys.stdout.flush()
            ifg_src = os.path.join(interferograms_src, ifg)
            ifg_dst = os.path.join(interferograms_dst, ifg)
            os.mkdir(ifg_dst)
            for f in files:
                file_src = os.path.join(ifg_src, f)
                file_dst = os.path.join(ifg_dst, f)
                if os.path.isfile(file_src):
                    shutil.copy(file_src, file_dst)
                else:
                    print(f"{file_src} doesn't exist.")
        print('\ndone.')

In [ ]:
stack_path = '/media/ly/文件/res2'
save_path = '/media/ly/file/sc1/isce_stack'
copy_files(stack_path, save_path)

## quickview

In [ ]:
from osgeo import gdal
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
plt.rcParams['figure.max_open_warning']=False
from matplotlib.colors import LightSource
import numpy as np
import glob
import os


def read_hgt(hgt_file):
    ds = gdal.Open(hgt_file, gdal.GA_ReadOnly)
    data = ds.GetRasterBand(1).ReadAsArray()
    ds = None
    return data


def view_unw(unws, hgt_file, out_dir):
    if unws:
        if not os.path.isdir(out_dir):
            os.mkdir(out_dir)
        for unw in unws:
            print(f"\rquickview of unws: {unws.index(unw) + 1}/{len(unws)}", end=" ", flush=True)
            ds = gdal.Open(unw, gdal.GA_ReadOnly)
            data = ds.GetRasterBand(2).ReadAsArray()
            ds = None
            fig,ax = plt.subplots(figsize=(11,10))
            hgt = read_hgt(hgt_file)
            ls = LightSource()
            hgt_shade = ls.shade(hgt, cmap=plt.get_cmap('gray'))
            ax.imshow(hgt_shade, zorder=0)
            m = np.max(np.abs(data))
            im = ax.imshow(data, cmap='jet', vmin=-m, vmax=m, zorder=1, alpha=0.7)
            fig.colorbar(im)
            ax.set_title('unwraped')
            unw_name = unw.split('/')[-2] + '_unw'
            fig.savefig(os.path.join(out_dir, unw_name), bbox_inches='tight')
        print('\ndone.')
        plt.close('all')
    else:
        print('no unws found.')

    
def view_ifg(ifgs, hgt_file, out_dir):
    if ifgs:
        if not os.path.isdir(out_dir):
            os.mkdir(out_dir)
        for ifg in ifgs:
            print(f"\rquickview of ifgs: {ifgs.index(ifg) + 1}/{len(ifgs)}", end=" ", flush=True)
            ds = gdal.Open(ifg, gdal.GA_ReadOnly)
            data = ds.GetRasterBand(1).ReadAsArray()
            ds = None
            fig,ax = plt.subplots(figsize=(11,10))
            hgt = read_hgt(hgt_file)
            ls = LightSource()
            hgt_shade = ls.shade(hgt, cmap=plt.get_cmap('gray'))
            ax.imshow(hgt_shade, zorder=0)
            im = ax.imshow(np.angle(data), cmap='jet', zorder=1, alpha=0.7)
            fig.colorbar(im)
            ax.set_title('interferogram')
            ifg_name = ifg.split('/')[-2] + '_ifg'
            fig.savefig(os.path.join(out_dir, ifg_name), bbox_inches='tight')
        print('\ndone.')
        plt.close('all')
    else:
        print('no ifgs found.')

    
def view_cor(cors, hgt_file, out_dir):
    if cors:
        if not os.path.isdir(out_dir):
            os.mkdir(out_dir)
        for cor in cors:
            print(f"\rquickview of cors: {cors.index(cor) + 1}/{len(cors)}", end=" ", flush=True)
            ds = gdal.Open(cor, gdal.GA_ReadOnly)
            data = ds.GetRasterBand(1).ReadAsArray()
            ds = None
            fig,ax = plt.subplots(figsize=(11,10))
            hgt = read_hgt(hgt_file)
            ls = LightSource()
            hgt_shade = ls.shade(hgt, cmap=plt.get_cmap('gray'))
            ax.imshow(hgt_shade, zorder=0)
            im = ax.imshow((data), cmap='gray', zorder=1, alpha=0.7)
            fig.colorbar(im)
            ax.set_title('coherence')
            cor_name = cor.split('/')[-2] + '_cor'
            fig.savefig(os.path.join(out_dir, cor_name), bbox_inches='tight')
        print('\ndone.')
        plt.close('all')
    else:
        print('no cors found.')

In [ ]:
interferograms_dir = '/media/ly/isce_stack/merged/interferograms'
unws = glob.glob(interferograms_dir + '/*/filt_fine.unw.vrt')
ifgs = glob.glob(interferograms_dir + '/*/filt_fine.int.vrt')
cors = glob.glob(interferograms_dir + '/*/filt_fine.cor.vrt')

hgt_file = '/media/ly/isce_stack/merged/geom_reference/hgt.rdr'

out_dir = '/media/ly/isce_stack/merged/quickview'

view_unw(unws, hgt_file, out_dir)
view_ifg(ifgs, hgt_file, out_dir)
view_cor(cors, hgt_file, out_dir)

# MintPy

## configure file

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from mintpy.objects import ifgramStack
from mintpy.utils import plot as pp, utils as ut
from mintpy import view, plot_network

prj_dir = '/media/ly/文件/sc2/mintpy6'
if not os.path.isdir(prj_dir):
    os.mkdir(prj_dir)

os.chdir(prj_dir)                        # work directory
prj_file = 'sc26.txt'                    # configure file

In [ ]:
config = """##-------------------------------- MintPy -----------------------------##
########## 1. Load Data (--load to exit after this step)
## load_data.py -H to check more details and example inputs.
mintpy.load.processor        = isce
##---------for ISCE only:
mintpy.load.metaFile         = ../isce_stack/reference/IW*.xml
mintpy.load.baselineDir      = ../isce_stack/baselines
##---------interferogram datasets:
mintpy.load.unwFile          = ../isce_stack/merged/interferograms/*/filt_*.unw
mintpy.load.corFile          = ../isce_stack/merged/interferograms/*/filt_*.cor
mintpy.load.connCompFile     = ../isce_stack/merged/interferograms/*/filt_*.unw.conncomp
##---------geometry datasets:
mintpy.load.demFile          = ../isce_stack/merged/geom_reference/hgt.rdr
mintpy.load.lookupYFile      = ../isce_stack/merged/geom_reference/lat.rdr
mintpy.load.lookupXFile      = ../isce_stack/merged/geom_reference/lon.rdr
mintpy.load.incAngleFile     = ../isce_stack/merged/geom_reference/los.rdr
mintpy.load.azAngleFile      = ../isce_stack/merged/geom_reference/los.rdr
mintpy.load.shadowMaskFile   = ../isce_stack/merged/geom_reference/shadowMask.rdr
mintpy.load.waterMaskFile    = None
##---------subset (optional):
mintpy.subset.lalo 			= 30.522521:30.915466,102.775720:103.124288

########## 2. modify_network
mintpy.network.keepMinSpanTree 		= yes  #[yes / no], auto for yes, keep interferograms in Min Span Tree network

########## 3. reference_point

########## 4. correct_unwrap_error (optional)

########## 5. invert_network
mintpy.networkInversion.minTempCoh  	= 0.5 #[0.0-1.0], auto for 0.7, min temporal coherence for mask

########## 6. correct_troposphere (optional and recommended)
mintpy.troposphericDelay.method         = height_correlation   #[pyaps / height_correlation / no], auto for pyaps

########## 7. deramp (optional)
mintpy.deramp                           = linear

########## 8. correct_topography (optional and recommended)

########## 9.1 residual_RMS (root mean squares for noise evaluation)

########## 9.2 reference_date

########## 10. velocity

########## 11.1 geocode (post-processing)

########## 11.2 google_earth (post-processing)

########## 11.3 hdfeos5 (post-processing)

########## 11.4 plot
"""
with open(prj_file, 'w+') as f:
    f.write(config)

## load_data

In [ ]:
!smallbaselineApp.py $prj_file --dostep load_data

In [ ]:
view.main('./inputs/ifgramStack.h5 20170120_20170213 --nrows 1 --ncols 3'.split())

In [ ]:
!info.py ./inputs/geometryRadar.h5 --compact

In [ ]:
# view.main('./inputs/geometryRadar.h5 --notick --noaxis'.split())

## modify_network

In [ ]:
!smallbaselineApp.py $prj_file --dostep modify_network

In [ ]:
plot_network.main(['./inputs/ifgramStack.h5'])

## reference_point

In [ ]:
!smallbaselineApp.py $prj_file --dostep reference_point

In [ ]:
!info.py ./inputs/ifgramStack.h5 | egrep 'REF_'

## correct_unwrap_error

In [ ]:
!smallbaselineApp.py $prj_file --dostep correct_unwrap_error

## invert_network

In [ ]:
!smallbaselineApp.py $prj_file --dostep invert_network

In [ ]:
view.main('timeseries.h5 --wrap --wrap-range -5 5 -u cm --notitle --notick --noaxis'.split())

In [ ]:
view.main('temporalCoherence.h5 --notick --noaxis --noverbose'.split())
view.main('maskTempCoh.h5 --notick --noaxis --noverbose'.split())

## correct_troposphere

In [ ]:
!smallbaselineApp.py $prj_file --dostep correct_troposphere

In [ ]:
view.main('timeseries_ERA5.h5 --wrap --wrap-range -5 5 -u cm --notitle --notick --noaxis'.split())

## deramp

In [ ]:
!smallbaselineApp.py $prj_file --dostep deramp

## correct_topography

In [ ]:
!smallbaselineApp.py $prj_file --dostep correct_topography

In [ ]:
view.main('timeseries_ERA5_ramp_demErr.h5 --notitle --wrap --wrap-range -5 5 --notick --noaxis'.split())

## residual_RMS

In [ ]:
!smallbaselineApp.py $prj_file --dostep residual_RMS

In [ ]:
!head rms_timeseriesResidual_ramp.txt

## reference_date

In [ ]:
!cat reference_date.txt    #auto selected optimal reference date.

In [ ]:
## plot the RMS value
from mintpy.timeseries_rms import plot_rms_bar
txtContent = np.loadtxt('./rms_timeseriesResidual_ramp.txt', dtype=bytes).astype(str)
rms_list = [float(i) for i in txtContent[:, 1]]
date_list = [i for i in txtContent[:, 0]]
fig, ax = plt.subplots(figsize=[10, 4])
ax = plot_rms_bar(ax, date_list, rms_list)
plt.show()

In [ ]:
!smallbaselineApp.py $prj_file --dostep reference_date

## velocity

In [ ]:
!smallbaselineApp.py $prj_file --dostep velocity

In [ ]:
!info.py velocity.h5 --compact

In [ ]:
view.main('velocityERA5.h5 --notick --noaxis --noverbose'.split())

## geocode

In [ ]:
!smallbaselineApp.py $prj_file --dostep geocode

In [ ]:
view.main('./geo/geo_velocity.h5 velocity --dem ./geo/geo_geometryRadar.h5 --figsize 12 12 --notick --noaxis'.split())

## google_earth

In [ ]:
!smallbaselineApp.py $prj_file --dostep google_earth

## save kmz

In [ ]:
# !save_kmz.py -h

In [ ]:
!save_kmz.py ./geo/geo_velocity.h5 -u mm  --noreference -v -60 60 -c jet -o ./geo/geo_velocity_mm

In [ ]:
# !save_kmz_timeseries.py -h

In [ ]:
!save_kmz_timeseries.py ./geo/geo_timeseries_ERA5_ramp_demErr.h5 --steps 20 5 1 --cutoff 0

## plot_smallbaselineApp

In [ ]:
# !plot_smallbaselineApp.sh